In [1]:
!pip3 install natsort

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from natsort import natsorted
import os
import glob
import cv2
import pydicom
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K

In [3]:
train = pd.read_csv('/kaggle/input/rsna-dataframe/train_data.csv')
train = train.astype({'study_id':'str','series_id':'str','instance_number':'str'})
train_des = pd.read_csv('/kaggle/input/rsna-dataframe/train_des.csv')
train_des = train_des.astype({'study_id':'str','series_id':'str'})

In [4]:
train_des

,study_id,series_id,series_description
0,100206310,1792451510,Sagittal T2/STIR
1,100206310,2092806862,Sagittal T1
2,100206310,1012284084,Axial T2
3,1002894806,801316590,Sagittal T2/STIR
4,1002894806,866293114,Sagittal T1
...,...,...,...
5362,991428866,2954790819,Sagittal T1
5363,991428866,3160509931,Axial T2
5364,992674144,1576603050,Sagittal T2/STIR
5365,992674144,1814811290,Sagittal T1


In [5]:
base_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
SagittalT2 = train_des[train_des['series_description']=='Sagittal T2/STIR'].reset_index(drop=True)
SagittalT2_path = base_dir + '/' + SagittalT2['study_id'] + '/' + SagittalT2['series_id'] + '/*.dcm'

In [6]:
def load_images(dir_path, s=0, ends=100):
    
    images = []
    path_list = []
    
    for p in tqdm(dir_path[s:ends]):
        path = natsorted(glob.glob(p))
        path_list += path
        
        for im in path:
            dcm = pydicom.dcmread(im)
            dcm = dcm.pixel_array
            dcm = (dcm - dcm.min()) / (dcm.max() - dcm.min() + 1e-6)*255.0
            dcm = dcm / 255.0
            dcm = cv2.resize(dcm,(224,224))
            images.append(dcm)
            
    base_label = base_dir + '/' + train['study_id'] + '/' + train['series_id'] + '/' + train['instance_number']+'.dcm'
    base_label = base_label.unique()
    
    labels = []
    lens = len(path_list)
    
    for i in range(lens):
        te = path_list[i] in base_label
        labels.append(te)
        
    enc = OneHotEncoder(sparse_output=False)
    label = enc.fit_transform(np.array(labels).reshape(-1,1))
            
    return np.array(images)[:,:,:,np.newaxis], label

In [7]:
train_image, train_label = load_images(SagittalT2_path, 0, 1000)

100%|██████████| 1000/1000 [07:12<00:00,  2.31it/s]


In [8]:
valid_image, valid_label = load_images(SagittalT2_path, 1000,1200)

100%|██████████| 200/200 [01:23<00:00,  2.38it/s]


In [9]:
def dense_block(x, num_layers, growth_rate):
    for i in range(num_layers):
        # Bottleneck Layer
        bn = layers.BatchNormalization()(x)
        relu = layers.ReLU()(bn)
        conv = layers.Conv2D(4 * growth_rate, (1, 1), padding='same', kernel_initializer='he_normal')(relu)

        # Composite Function
        bn = layers.BatchNormalization()(conv)
        relu = layers.ReLU()(bn)
        conv = layers.Conv2D(growth_rate, (3, 3), padding='same', kernel_initializer='he_normal')(relu)
        
        # Concatenate input with output of this layer
        x = layers.Concatenate()([x, conv])
    return x

def transition_layer(x, reduction):
    # Batch Normalization and ReLU
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # 1x1 Convolution
    num_filters = int(K.int_shape(x)[-1] * reduction)
    x = layers.Conv2D(num_filters, (1, 1), padding='same', kernel_initializer='he_normal')(x)
    
    # Average Pooling
    x = layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
    
    return x

def build_densenet(inputs, depth, growth_rate, reduction=0.5):
    
    # 初期コンボリューションレイヤー
    x = layers.Conv2D(2 * growth_rate, (7, 7), strides=(2, 2), padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    # Dense BlockとTransition Layerの構築
    num_dense_blocks = (depth - 4) // 6
    for i in range(3):  # Typically, DenseNet has 3 dense blocks.
        x = dense_block(x, num_dense_blocks, growth_rate)
        if i != 2:  # 最後のDense Blockの後にTransition Layerは不要
            x = transition_layer(x, reduction)
    
    # グローバル平均プーリングと全結合レイヤー
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    return x

In [10]:
def CNN_model(input_shape, num_classes, depth, growth_rate):
    
    inputs = Input(shape=input_shape)
    x = build_densenet(inputs, depth, growth_rate)
    
    outputs = layers.Dense(num_classes, activation='sigmoid', name='output')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [11]:
# モデルの構築
input_shape = (224, 224, 1)  # 画像のサイズ
num_classes = 2  # クラス数
depth = 40  # ネットワークの深さ
growth_rate = 12  # グロースレート

In [12]:
scs_model = CNN_model(input_shape, num_classes, depth, growth_rate)
scs_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
nfn_model = CNN_model(input_shape, num_classes, depth, growth_rate)
nfn_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
scs_history = scs_model.fit(train_image, train_label, 
                            epochs=20, batch_size=64,
                            validation_data=(valid_image, valid_label))

Epoch 1/20


I0000 00:00:1729003539.854989      71 service.cc:145] XLA service 0x795d000f9600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729003539.855040      71 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  2/267 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - accuracy: 0.9609 - loss: 0.4733   

I0000 00:00:1729003568.662576      71 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


267/267 ━━━━━━━━━━━━━━━━━━━━ 101s 186ms/step - accuracy: 0.9323 - loss: 0.2120 - val_accuracy: 0.9222 - val_loss: 0.2905
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9446 - loss: 0.1235 - val_accuracy: 0.9499 - val_loss: 0.1055
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9490 - loss: 0.1106 - val_accuracy: 0.9475 - val_loss: 0.1212
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9502 - loss: 0.1073 - val_accuracy: 0.9511 - val_loss: 0.1062
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9574 - loss: 0.0923 - val_accuracy: 0.9446 - val_loss: 0.1202
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9580 - loss: 0.0881 - val_accuracy: 0.9525 - val_loss: 0.0981
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9610 - loss: 0.0863 - val_accuracy: 0.9337 - val_loss: 0.2097
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9634 - loss: 0.0844 - val_accur

In [15]:
test_image, test_label = load_images(SagittalT2_path, 1200, 1300)

100%|██████████| 100/100 [00:44<00:00,  2.24it/s]


In [16]:
pred = scs_model.predict(test_image)
ac = np.argmax(pred, axis=1)
test = np.argmax(test_label, axis=1)

55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step


In [17]:
a = metrics.Accuracy()
a.update_state(ac,test)
print(a.result())

l = metrics.BinaryCrossentropy()
l.update_state(pred,test_label)
print(l.result())

tf.Tensor(0.9397936, shape=(), dtype=float32)
tf.Tensor(1.1723702, shape=(), dtype=float32)


In [18]:
scs_model.save('/kaggle/working/scs_extraction_model.h5')